# Install Dependencies

In [ ]:
!pip install gradio

# Load Dataset

In [2]:
from sklearn.model_selection import train_test_split
import pandas as pd


tweets = pd.read_csv('train.csv', sep=',')
selected_columns = ['text', 'target']
twitter_df = tweets[selected_columns]

In [3]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Conv1D, MaxPooling1D, Dense, GlobalMaxPooling1D
from keras.preprocessing.text import Tokenizer

In [4]:
X = twitter_df['text']
y = twitter_df['target']

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Tokenize Datapoints

In [5]:
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

max_words = 5000
max_len = 100
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(X_train)
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_val_seq = tokenizer.texts_to_sequences(X_val)
X_train_padded = pad_sequences(X_train_seq, maxlen=max_len)
X_val_padded = pad_sequences(X_val_seq, maxlen=max_len)

# LSTM

In [6]:
lstm_model = Sequential()
lstm_model.add(Embedding(max_words, 128, input_length=max_len))
lstm_model.add(LSTM(64, dropout=0.2, recurrent_dropout=0.2))
lstm_model.add(Dense(1, activation='sigmoid'))
lstm_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
lstm_model.fit(X_train_padded, y_train, validation_data=(X_val_padded, y_val), epochs=5, batch_size=64)

Epoch 1/5
96/96 [==============================] - 28s 252ms/step - loss: 0.5707 - accuracy: 0.7000 - val_loss: 0.4701 - val_accuracy: 0.7807
Epoch 2/5
96/96 [==============================] - 24s 253ms/step - loss: 0.3607 - accuracy: 0.8470 - val_loss: 0.4521 - val_accuracy: 0.8050
Epoch 3/5
96/96 [==============================] - 23s 241ms/step - loss: 0.2760 - accuracy: 0.8905 - val_loss: 0.5011 - val_accuracy: 0.7932
Epoch 4/5
96/96 [==============================] - 23s 236ms/step - loss: 0.2174 - accuracy: 0.9205 - val_loss: 0.5228 - val_accuracy: 0.7846
Epoch 5/5
96/96 [==============================] - 24s 252ms/step - loss: 0.1872 - accuracy: 0.9322 - val_loss: 0.5939 - val_accuracy: 0.7643


In [7]:
lstm_pred = (lstm_model.predict(X_val_padded) > 0.5).astype(int).flatten()
print("\nLSTM Accuracy:", accuracy_score(y_val, lstm_pred))
print("\nLSTM Classification Report:")
print(classification_report(y_val, lstm_pred))

48/48 [==============================] - 1s 23ms/step

LSTM Accuracy: 0.7642810242941562

LSTM Classification Report:
              precision    recall  f1-score   support

           0       0.81      0.77      0.79       874
           1       0.71      0.75      0.73       649

    accuracy                           0.76      1523
   macro avg       0.76      0.76      0.76      1523
weighted avg       0.77      0.76      0.77      1523



# CNN

In [8]:
cnn_model = Sequential()
cnn_model.add(Embedding(max_words, 128, input_length=max_len))
cnn_model.add(Conv1D(64, 5, activation='relu'))
cnn_model.add(MaxPooling1D(pool_size=4))
cnn_model.add(GlobalMaxPooling1D())
cnn_model.add(Dense(1, activation='sigmoid'))
cnn_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
cnn_model.fit(X_train_padded, y_train, validation_data=(X_val_padded, y_val), epochs=5, batch_size=64)

Epoch 1/5
96/96 [==============================] - 8s 74ms/step - loss: 0.6070 - accuracy: 0.6790 - val_loss: 0.5062 - val_accuracy: 0.7833
Epoch 2/5
96/96 [==============================] - 6s 58ms/step - loss: 0.3877 - accuracy: 0.8369 - val_loss: 0.4482 - val_accuracy: 0.7919
Epoch 3/5
96/96 [==============================] - 7s 71ms/step - loss: 0.2654 - accuracy: 0.8966 - val_loss: 0.4836 - val_accuracy: 0.8037
Epoch 4/5
96/96 [==============================] - 6s 58ms/step - loss: 0.1916 - accuracy: 0.9315 - val_loss: 0.5332 - val_accuracy: 0.7886
Epoch 5/5
96/96 [==============================] - 7s 71ms/step - loss: 0.1376 - accuracy: 0.9542 - val_loss: 0.6050 - val_accuracy: 0.7807


In [9]:
cnn_pred = (cnn_model.predict(X_val_padded) > 0.5).astype(int).flatten()
print("\nLSTM Accuracy:", accuracy_score(y_val, cnn_pred))
print("\nLSTM Classification Report:")
print(classification_report(y_val, cnn_pred))

48/48 [==============================] - 0s 8ms/step

LSTM Accuracy: 0.7806959947472094

LSTM Classification Report:
              precision    recall  f1-score   support

           0       0.80      0.82      0.81       874
           1       0.75      0.72      0.74       649

    accuracy                           0.78      1523
   macro avg       0.78      0.77      0.77      1523
weighted avg       0.78      0.78      0.78      1523



# GRU

In [10]:
from keras.layers import Embedding, GRU, Bidirectional, Dense, GlobalMaxPooling1D

gru_model = Sequential()
gru_model.add(Embedding(max_words, 128, input_length=max_len))
gru_model.add(GRU(64, dropout=0.2, recurrent_dropout=0.2))
gru_model.add(Dense(1, activation='sigmoid'))
gru_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
gru_model.fit(X_train_padded, y_train, validation_data=(X_val_padded, y_val), epochs=5, batch_size=64)

Epoch 1/5
96/96 [==============================] - 26s 232ms/step - loss: 0.5916 - accuracy: 0.6905 - val_loss: 0.4760 - val_accuracy: 0.7951
Epoch 2/5
96/96 [==============================] - 21s 215ms/step - loss: 0.3639 - accuracy: 0.8430 - val_loss: 0.4836 - val_accuracy: 0.7735
Epoch 3/5
96/96 [==============================] - 22s 229ms/step - loss: 0.2798 - accuracy: 0.8857 - val_loss: 0.5310 - val_accuracy: 0.7452
Epoch 4/5
96/96 [==============================] - 22s 230ms/step - loss: 0.2245 - accuracy: 0.9138 - val_loss: 0.5997 - val_accuracy: 0.7833
Epoch 5/5
96/96 [==============================] - 21s 216ms/step - loss: 0.1874 - accuracy: 0.9281 - val_loss: 0.6646 - val_accuracy: 0.7590


In [11]:
gru_pred = (gru_model.predict(X_val_padded) > 0.5).astype(int).flatten()
print("\nGRU Accuracy:", accuracy_score(y_val, gru_pred))
print("\nGRU Classification Report:")
print(classification_report(y_val, gru_pred))

48/48 [==============================] - 1s 20ms/step

GRU Accuracy: 0.7590282337491793

GRU Classification Report:
              precision    recall  f1-score   support

           0       0.79      0.79      0.79       874
           1       0.72      0.71      0.72       649

    accuracy                           0.76      1523
   macro avg       0.75      0.75      0.75      1523
weighted avg       0.76      0.76      0.76      1523



# Bi-Directional RNN

In [12]:
bidir_model = Sequential()
bidir_model.add(Embedding(max_words, 128, input_length=max_len))
bidir_model.add(Bidirectional(GRU(64, dropout=0.2, recurrent_dropout=0.2)))
bidir_model.add(Dense(1, activation='sigmoid'))
bidir_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
bidir_model.fit(X_train_padded, y_train, validation_data=(X_val_padded, y_val), epochs=5 , batch_size=64)

Epoch 1/5
96/96 [==============================] - 48s 425ms/step - loss: 0.5790 - accuracy: 0.6916 - val_loss: 0.4591 - val_accuracy: 0.7892
Epoch 2/5
96/96 [==============================] - 40s 416ms/step - loss: 0.3592 - accuracy: 0.8476 - val_loss: 0.4575 - val_accuracy: 0.8030
Epoch 3/5
96/96 [==============================] - 40s 417ms/step - loss: 0.2780 - accuracy: 0.8862 - val_loss: 0.5018 - val_accuracy: 0.7925
Epoch 4/5
96/96 [==============================] - 40s 418ms/step - loss: 0.2213 - accuracy: 0.9130 - val_loss: 0.5715 - val_accuracy: 0.7715
Epoch 5/5
96/96 [==============================] - 40s 418ms/step - loss: 0.1919 - accuracy: 0.9302 - val_loss: 0.6606 - val_accuracy: 0.7676


In [13]:
bidir_pred = (bidir_model.predict(X_val_padded) > 0.5).astype(int).flatten()
print("\nBidirectional RNN Accuracy:", accuracy_score(y_val, bidir_pred))
print("\nBidirectional RNN Classification Report:")
print(classification_report(y_val, bidir_pred))

48/48 [==============================] - 2s 34ms/step

Bidirectional RNN Accuracy: 0.767564018384767

Bidirectional RNN Classification Report:
              precision    recall  f1-score   support

           0       0.80      0.80      0.80       874
           1       0.73      0.72      0.73       649

    accuracy                           0.77      1523
   macro avg       0.76      0.76      0.76      1523
weighted avg       0.77      0.77      0.77      1523



# Gradio

In [14]:
from keras.preprocessing.sequence import pad_sequences

def dl_classify_tweet(tweet, model, tokenizer, max_len=100):
    seq = tokenizer.texts_to_sequences([tweet])
    padded = pad_sequences(seq, maxlen=max_len)
    prediction = model.predict(padded)
    class_label = 'Disaster' if prediction[0][0] >= 0.5 else 'Not Disaster'

    return class_label

In [15]:
import gradio as gr

def predict(model, tweet):
    return dl_classify_tweet(tweet, model, tokenizer)


def classify_tweet(tweet, expected):
    results = {}
    justifications = {}

    models = {
        'LSTM': lstm_model,
        'CNN': cnn_model,
        'GRU': gru_model,
        'Bidirectional RNN': bidir_model,
    }
    for model_name, model in models.items():
        prediction = predict(model, tweet)
        match = 'green' if prediction == expected else 'red'
        results[model_name] = f"<div style='color: {match}; border:2px solid {match}; padding:5px; margin:2px;'>{model_name}: {prediction}/{expected}</div>"

    return list(results.values())

# Set up the Gradio interface
iface = gr.Interface(
    fn=classify_tweet,
    inputs=[
        gr.Textbox(lines=2, placeholder="Enter your tweet here..."),
        gr.Radio(choices=["Disaster", "Not Disaster"], label="Expected Classification")
    ],
    outputs=[
        gr.HTML(label=model_name) for model_name in [
            'LSTM', 'CNN', 'GRU', 'Bidirectional RNN'
        ]

    ]
)

# Run the Gradio app
iface.launch()


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://f285d8f8982463d6af.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
